<a href="https://colab.research.google.com/github/xvxvdee/CPS870-Research-Project/blob/main/ResearchProject_Dee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
import os

In [ ]:
!pip install fpdf

!pip install nltk
# !pip install spacy
# #!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg/en_core_web_lg.tar.gz
# !pip install pyresparser
# # spaCy
# !python -m spacy download en_core_web_lg

# nltk
!python -m nltk.downloader stopwords
!python -m nltk.downloader punkt
!python -m nltk.downloader averaged_perceptron_tagger
!python -m nltk.downloader universal_tagset
!python -m nltk.downloader wordnet
!python -m nltk.downloader brown
!python -m nltk.downloader maxent_ne_chunker

!pip install resume-pyresparser

**KAGGLE**

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/Shareddrives/CPS870/Final Project/Kaggle"

In [ ]:
os.chdir("/content/gdrive/Shareddrives/CPS870/Final Project/text_pdf")

In [ ]:
!kaggle datasets download -d oo7kartik/resume-text-batch

In [ ]:
!unzip \*.zip  && rm *.zip

**GETTING DATA**

In [ ]:
arr = []
for root, dirs, files in os.walk("/content/gdrive/Shareddrives/CPS870/Final Project/resume_data"):
    for file in files:      
      arr.append(os.path.join(root, file))

In [ ]:
print(len(arr))

15066


In [ ]:
# !pip install https://github.com/MartinoMensio/spacy-universal-sentence-encoder/releases/download/v0.4.3/en_use_md-0.4.3.tar.gz#en_use_md-0.4.3
# nlp = spacy.load('en_use_md')

**CONVERT AND SAVE AS PDF ✅** 

In [ ]:
import numpy as np
from fpdf import FPDF
import re

#needs to be changed into a for loop
for i in range(0,len(arr)):
  pdf = FPDF()  
  pdf.add_page() 
  pdf.set_font("Arial", size = 15) 

  text_file = open(arr[i], "r") 
  data = text_file.read()

  # #encode() method
  strencode = data.encode("ascii", "ignore") # removing ascii that cannot be parsed
  
  # #decode() method
  strdecode = strencode.decode()

  setup = strdecode.split("\n")
  lines = [x for x in setup if len(x)>0]

  res = "\n".join(lines)

  pdf.set_font("Arial", size=12)

  for x in lines:
    l = x
    if "\t" in l: l.replace("\t", "  ")
    pdf.multi_cell(190, 10, txt =l) 

  pdf.output("resume_"+str(i)+".pdf") # saving file
  print("Resume "+str(i)+" has been saved.")


In [ ]:
os.chdir("/content/gdrive/Shareddrives/CPS870/Final Project/resume parser results")

In [ ]:
pdf = []
for root, dirs, files in os.walk("/content/gdrive/Shareddrives/CPS870/Final Project/text_pdf"):
    for file in files:      
      pdf.append(os.path.join(root, file))

In [ ]:
from pyresparser import ResumeParser
import numpy as np
for i in range(0,len(pdf)):
  data = ResumeParser(pdf[i]).get_extracted_data()
  np.save("resume_dict_"+str(i)+".npy",data)
  print("dct saved for resume "+str(i))

**CLEANING DATA**

In [ ]:
res = []
for root, dirs, files in os.walk("/content/gdrive/Shareddrives/CPS870/Final Project/resume parser results"):
    for file in files:      
      res.append(os.path.join(root, file))

In [ ]:
print(len(res))

15225


 **DOC SIM**

In [ ]:
import spacy
!pip install https://github.com/MartinoMensio/spacy-universal-sentence-encoder/releases/download/v0.4.3/en_use_md-0.4.3.tar.gz#en_use_md-0.4.3
nlp = spacy.load('en_use_md')

Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB



In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stop=stopwords.words('english')
avoid = ["email","objective","mobile","languages","place:","nationality","achivements","qualifications","strengths:","experience","father's","name","marital","status","gender","religion","date:",'role:']

def words_Resume(data):
  res = []
  for i in data:
    if i in stop: continue
    if len(i)>1:
      lst = i.split(" ")
      for j in lst:
        if len(j)>2 and j not in stop and j not in avoid:res.append(j.lower())
    elif i not in stop and i not in avoid : res.append(i.lower())
  return res

def get_role(num):
  dict_res1 = np.load(res[num],allow_pickle=True ) 
  test = dict_res1.item()
  try: 
    deg = [x.replace("\n","") for x in test['degree']]
    return [test['designation'],deg]
  except: 
    return []

def get_skills(num):
  dict_res1 = np.load(res[num],allow_pickle=True ) 
  test = dict_res1.item()
  # print(test)
  # print("---")
  try: 
    return test['skills']
  except: 
    return []

def get_experience(num):
  dict_res1 = np.load(res[num],allow_pickle=True ) 
  test = dict_res1.item()
  try:
    return test['experience']
  except:
    return []

def check_unicode(word):
  w=""
  for i in word:
    if i.isdigit(): continue
    else: w+=i
  try:
    return w.replace("cid","")
  except: return w

def get_comparsionLst(skills,experience):
  res = []
  ex = [re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", x) for x in " ".join(experience).split(" ") if len(x)!=0 and x not in stop]
  for i in skills: 
    if i not in stop:  res.append(i.lower())
  for i in ex: 
    word = check_unicode(i.lower())
    if word!='career' and word!="objective" and len(word)!=0 and word not in stop:res.append( re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", word))
  return res

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
os.chdir("/content/gdrive/Shareddrives/CPS870/Final Project/Doc Sim")

In [ ]:
import nltk
import numpy as np
import logging as debug

from nltk.corpus import stopwords
nltk.download('stopwords')
stop=stopwords.words('english')

sim = []
for i in range(17,5000):
  sim=[]
  s=[]

  exp=get_experience(i)
  skill = get_skills(i)

  if skill == None or exp == None: continue

  res1 = set(get_comparsionLst(skill,exp))
  role_1 =get_role(i)
  doc1 =nlp(" ".join(res1))

  for j in range(0,500):
    if i!=j:
      exp_2 = get_experience(j)
      skill_2 = get_skills(j)
      if exp_2== None or skill_2 == None:continue
      res2 = set(get_comparsionLst(skill_2,exp_2))
      role_2=get_role(j)
      doc2 =nlp(" ".join(res2))

      similar = set([x for x in res1 if x in res2 and x not in avoid])
      similarity = doc1.similarity(doc2)

      sim.append([similarity,i,j,role_1,role_2,similar])
      if j%100==0:print(j)

  s=sorted(sim,key=lambda x: x[1])
  stats = s[-5:]+s[:2]
  np.save("resume_sim_"+str(i)+".npy",stats)
  print("resume_sim_"+str(i)+" completed")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100
200
300
400


/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Streaming output truncated to the last 5000 lines.
100
200
300
400
resume_sim_3651 completed
100
200
300
400
resume_sim_3654 completed
100
200
300
400
resume_sim_3655 completed
100
200
300
400
resume_sim_3656 completed
100
200
300
400
resume_sim_3658 completed
100
200
300
400
resume_sim_3660 completed
100
200
300
400
resume_sim_3662 completed
100
200
300
400
resume_sim_3663 completed
100
200
300
400
resume_sim_3664 completed
100
200
300
400
resume_sim_3665 completed
100
200
300
400
resume_sim_3666 completed
100
200
300
400
resume_sim_3667 completed
100
200
300
400
resume_sim_3668 completed
100
200
300
400
resume_sim_3669 completed
100
200
300
400
resume_sim_3671 completed
100
200
300
400
resume_sim_3675 completed
100
200
300
400
resume_sim_3677 completed
100
200
300
400
resume_sim_3679 completed
100
200
300
400
resume_sim_3681 completed
100
200
300
400
resume_sim_3683 completed
100
200
300
400
resume_sim_3686 completed
100
200
300
400
resume_sim_3687 completed
100
200
300
400
resume_si

In [ ]:
s=sorted(sim,key=lambda x: x[1])
s[:2]

[[0.10990888356969905,
  4,
  488,
  [['Asst.HR Manager', 'Senior .HR Executive'],
   ['Master of Business Administration(cid:9)']],
  [],
  set()],
 [0.12586583450493583,
  4,
  72,
  [['Asst.HR Manager', 'Senior .HR Executive'],
   ['Master of Business Administration(cid:9)']],
  [],
  {'administration',
   'english',
   'etc',
   'operating systems',
   'provide',
   'resources',
   'support'}]]

In [ ]:
stats_res = []
for root, dirs, files in os.walk("/content/gdrive/Shareddrives/CPS870/Final Project/Doc Sim"):
    for file in files:      
      stats_res.append(os.path.join(root, file))

In [ ]:
all_stats = []
build =[]

In [ ]:
import numpy as np
print(len(stats_res))
for i in range(len(stats_res)):
  if 'txt' in stats_res[i]:continue
  ar = np.load(stats_res[i],allow_pickle=True)
  build.append(ar)
  print(i)
for i in build:
  for j in i:
    all_stats.append(j)

In [ ]:
srt=sorted(all_stats,key=lambda x: x[0])
top=[[x[0],x[3],x[4],x[5]]for x in srt[-25:]]
bottom=[[x[0],x[3],x[4],x[5]] for x in srt[:56] if len(x[3])!=0]


In [ ]:
#!pip install tabulate
from tabulate import tabulate
print(tabulate(top, headers=['Sim Score', 'R1 Degree/Current Position','R2 Degree/Current Position','Key Wordss'], tablefmt='orgtbl'))

|   Sim Score | R1 Degree/Current Position                                                                                                                                                             | R2 Degree/Current Position                                                                                                                                                                                                      | Key Wordss                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   